<a href="https://colab.research.google.com/github/PigStep/furniture-store-extraction/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

#For parsing
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
urls_df = pd.read_csv("URL_list.csv")
urls_df.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


In [3]:
url_series = pd.Series(urls_df["max(page)"]) #To make code easier
url_series.head()

,max(page)
0,https://www.factorybuys.com.au/products/euro-t...
1,https://dunlin.com.au/products/beadlight-cirrus
2,https://themodern.net.au/products/hamar-plant-...
3,https://furniturefetish.com.au/products/oslo-o...
4,https://hemisphereliving.com.au/products/


# Предобработка данных
Сохраним только открывающиеся ссылки

In [4]:
def check_is_url_parsing(url):
  headers = {"User-Agent": "Mozilla/5.0"}
  try:
      response = requests.get(url, headers=headers, timeout=10)
      response.raise_for_status() #check for html error
      soup = BeautifulSoup(response.text, "html.parser")

      return True

  except requests.exceptions.RequestException as e:
    print(f"Attemp to URL falied: {e}")
    return False

In [5]:
def create_good_urls_csv():
  """Создает файл 'ParsingURL_list.csv' с посещаемыми ссылками"""
  good_urls =[]
  for url in url_series:
    if(check_is_url_parsing(url)):
      good_urls.append(url)

  urls = pd.Series(good_urls, name="url")
  urls.to_csv("ParsingURL_list.csv")
  print("Dataset have been created and saved")

In [6]:
# create_good_urls_csv()

In [7]:
# Пересоздадим серию c очищенными ссылками
url_series = pd.read_csv("ParsingURL_list.csv")
url_series = pd.Series(url_series["url"])

#Parsing data from sites

In [28]:
def extract_top_product_names(url, max_length=80, min_length=0, top_n=5):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    blacklist_tags = {"footer", "nav", "script", "style", "noscript", "form", "aside", "ul", "li"}
    candidates = {}

    def add_candidate(text, source):
        text = text.strip()
        if "$" in text:
          return
        if min_length < len(text) <= max_length:
            candidates[text] = source

    # 1. h1
    h1 = soup.find("h1")
    if h1:
        add_candidate(h1.get_text(), "h1")

    # 2. h2 и h3
    for tag in soup.find_all(["h2", "h3"]):
        if tag.find_parent(blacklist_tags): continue
        add_candidate(tag.get_text(), tag.name)

    # 3. По ключевым словам в class
    class_keywords = ["product__", "product-","title"]
    for tag in soup.find_all(True):  # True = все теги
        classes = tag.get("class")
        if not classes:
            continue
        if any(any(k in cls.lower() for k in class_keywords) for cls in classes):
            if tag.find_parent(blacklist_tags): continue
            add_candidate(tag.get_text(), f"class={','.join(classes)}")

    # Уникальные, по длине (сначала длинные — как более информативные)
    sorted_candidates = sorted(candidates.items(), key=lambda x: len(x[0]), reverse=True)

    return sorted_candidates[:top_n]


In [116]:
def create_list_for_annotations():
  with open("annote.txt", "w", encoding="utf-8") as file:
    for indx in range(0,100):
      url = url_series[indx]

      for i, (text, tag) in enumerate(extract_top_product_names(url,max_length=100,top_n=10)):
        text = text.replace("\n","")
        file.write(f'{text} | ')
      file.write('\n\n')

    print("saving complete in 'annote.txt'")

# create_list_for_annotations